In [1]:
!pip install ta

  Running setup.py bdist_wheel for ta ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/de/ac/09/28855b628633d42061f670c59df6877a1dbdf70d04c985bc18
Successfully built ta
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install datetime

    100% |████████████████████████████████| 61kB 2.7MB/s ta 0:00:011
    100% |████████████████████████████████| 174kB 38.4MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install pandas_datareader

    100% |████████████████████████████████| 122kB 4.4MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
import ta as ta
import warnings
import sys, os
import datetime

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


In [6]:
UST = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/USTREASURY-YIELD.csv'
FRED_PERMIT = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/FRED-PERMIT.csv'
ISMVS_GDP = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/ismvsgdp.csv'
USLGI = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/USLgI.csv'
INVERSION = 'https://raw.githubusercontent.com/black-swan-2/Black-Swan-2-Models/master/Data/inversion.csv'


In [7]:
def get_data():
    """
    Downloads and wrangles data for BlackSwan 2.0 VIX modeling. 
  
    Combines the two index datasets and creates a 
    new DataFrame (df) that contains the target feature (Trading Days with 3 
    STDV shifts based on a rolling window)
  
    :input: vix  => CBOE VIX Historical Data
    :input: gspc => GSPC S&P 500 index with matching start date to VIX
  
    :return: df  => cominbation VIX/GSPC Dataset with Target Classifier 
    """
  
    ############### Getting Data ##################
  
    vix = (data.DataReader('^VIX', 
                           "yahoo", 
                           start='1990-1-02', 
                           end='2019-5-31')
           .drop(columns = ['Volume', 'Adj Close']))
  
    gspc = data.DataReader('^GSPC', 
                           "yahoo", 
                           start='1990-1-02',
                           end='2019-5-31')
    
    treasury = (pd.read_csv(UST)
                .sort_values(by = 'Date')
                .drop(columns=['1 MO', '2 MO', '20 YR']))
    
    fred = pd.read_csv(FRED_PERMIT)
    
    ism = pd.read_csv(ISMVS_GDP).drop(index = [0,1])
    
    lag = pd.read_csv(USLGI)
    
    inv = (pd.read_csv(INVERSION)
           [['Date','3m1s_inversion','3m2s_inversion',
             '2s10s_inversion','2s30s_inversion']])
  
    ############### Wrangling Data #################
    
    # Rename the Columns
    vix.columns      = ['vix_high', 'vix_low', 'vix_open', 'vix_close']
    gspc.columns     = ['gspc_high', 'gspc_low', 'gspc_open',
                        'gspc_close','gspc_volume','gspc_adj_close']
  
    # Join the VIX and GSPC
    df = vix.join(gspc)
  
    # Pull Date columns out of the index
    df = df.reset_index()
    
    # Merge DF with the Treasury Data on the Date Feature
    # Date needs to be converted to Datetime format to match df['Date']
    treasury['Date'] = pd.to_datetime(treasury['Date'],
                                      infer_datetime_format=True)
    
    df = pd.merge(df, treasury, how='inner', on='Date')
    
    # Merge DF with the Inversion Features Engineered by Damerei
    # Sort and create key for merging
    inv = inv[inv['Date'] < '2019-06-01'].sort_values(by='Date')
    inv['Date'] = pd.to_datetime(inv['Date'], infer_datetime_format=True)
    
    df = pd.merge(df, inv, how='inner', on='Date')
    
    # A new feature is needed to act as a key for the following features
    df['y/m'] = df['Date'].map(lambda x: x.strftime('%Y-%m'))
    
    # Line up the Date column with the new key for the FRED Data
    fred = fred[fred['Date'] > '1989-12-01'].sort_values(by='Date')
    fred['Date'] = pd.to_datetime(fred['Date'], infer_datetime_format=True)
    fred['Date'] =  fred['Date'].map(lambda x: x.strftime('%Y-%m'))
    
    # Line up the Date column iwth the new key for the ISM/GDP Data
    ism = ism[ism['ticker'] > '1989-12-01'].sort_values(by='ticker')
    ism['ticker'] = pd.to_datetime(ism['ticker'], infer_datetime_format=True)
    ism['ticker'] =  ism['ticker'].map(lambda x: x.strftime('%Y-%m'))
    # Add most recent ISM data
    ism = (ism.append(pd.DataFrame([['2019-04', 52.8, np.nan]
                                   ,['2019-05', 52.1,np.nan]]
                                   ,columns=ism.columns))
           .reset_index()
           .drop(columns='index'))
    #Add most recent GDP Value
    ism['GDP CURY Index'].iloc[351] = 5.0
    # Fill in Quarterly GDP values
    ism = ism.fillna(method='ffill').fillna(method='bfill')
    
    # Wrangle the date column for the Lagging Index Dataframe
    lag['Date'] = pd.to_datetime(lag['Date'], format='%b-%y')
    #Create a pivot point around 2019 to accomodate the 2-digit year format
    for i in np.arange(len(lag)):
        if lag['Date'][i].year > 2019:
            lag['Date'][i] = lag['Date'][i].replace(year=lag['Date'][i].year-100)
        else:
            pass
    # Map to the y/m key format
    lag['Date'] =  lag['Date'].map(lambda x: x.strftime('%Y-%m'))
    #Slice the dataframe to relevant time period
    lag = lag[lag['Date'] > '1989-12']
    

    # Create New Columns with a single iteration using list comprehension
    f  = []
    m  = []
    g  = []
    ll = []
    lg = []

    for i in np.arange(len(df)):
        # Merge the New Private Housing Units Authorized by Building Permits (FRED)
        f.append(float(fred.loc[fred['Date'] == df['y/m'][i]]['Value']
                       .values
                       .tolist()[0]))
        
        # Merge the ISM NAPMPMI Index
        m.append(float(ism.loc[ism['ticker'] == df['y/m'][i]]['NAPMPMI Index']
                       .values
                       .tolist()[0]))
        
        # Merge the GDP CURY Index
        g.append(float(ism.loc[ism['ticker'] == df['y/m'][i]]['GDP CURY Index']
                       .values
                       .tolist()[0]))
        
            # Merge the Lagging Index Level Column
        ll.append(float(lag.loc[lag['Date'] == df['y/m'][i]]['Level']
                        .values
                        .tolist()[0]))
        
        # Merge the Lagging Index Growth Column
        lg.append(float(lag.loc[lag['Date'] == df['y/m'][i]]['Growth']
                        .values
                        .tolist()[0]))
    
    df['lag_index_level'] = ll
    df['lag_index_growth'] = lg
    df['fred'] = f
    df['ism'] = m
    df['gdp_cury'] = g
    
  
    ############### Momemntum Feature Engineering ################
  
    # Awesome Oscillator
    df['mom_ao']=ta.momentum.ao(df['gspc_high'],
                                df['gspc_low'],
                                s=5,len=34,
                                fillna=True)

    # Money Flow Index
    df['mom_mf']=ta.momentum.money_flow_index(df['gspc_high'],
                                              df['gspc_low'],
                                              df['gspc_close'],
                                              df['gspc_volume'],
                                              n=14,fillna=True)
  
    # Relative Strength Index
    df['mom_rsi'] = ta.momentum.rsi(df['gspc_close'],
                                    n=14,
                                    fillna=True)
  
    # Stochasitc Oscillator
    df['mom_stoch']=ta.momentum.stoch(df['gspc_high'],
                                      df['gspc_low'],
                                      df['gspc_close'],
                                      n=14,
                                      fillna=True)
  
    # Stochasitc Signal
    df['mom_st_sig']=ta.momentum.stoch_signal(df['gspc_high'],
                                              df['gspc_low'],
                                              df['gspc_close'],
                                              n=14,
                                              d_n=3,
                                              fillna=True)
  
    # True Strength Indicator
    df['mom_tsi'] = ta.momentum.tsi(df['gspc_close'],
                                    r=25,
                                    s=13,
                                    fillna=True)
  
    # Ultimate Oscillator
    df['mom_uo'] = ta.momentum.uo(df['gspc_high'],
                                  df['gspc_low'],
                                  df['gspc_close'], 
                                  s=7, 
                                  m=14, 
                                  len=28, 
                                  ws=4.0, 
                                  wm=2.0, 
                                  wl=1.0,
                                  fillna=True)
  
    # Williams %R
    df['mom_wr']=ta.momentum.wr(df['gspc_high'],
                                df['gspc_low'],
                                df['gspc_close'],
                                lbp=14,fillna=True)
  
    ############### Volume Feature Engineering ####################
  
    # Accumulation/Distribution Index
    df['vol_adi']=ta.volume.acc_dist_index(df['gspc_high'],
                                           df['gspc_low'],
                                           df['gspc_close'],
                                           df['gspc_volume'],
                                           fillna=True)
  
    # Chaikin Money Flow
    df['vol_cmf'] = ta.volume.chaikin_money_flow(df['gspc_high'],
                                                 df['gspc_low'],
                                                 df['gspc_close'],
                                                 df['gspc_volume'],
                                                 n=20,fillna=True)
  
    # Ease of Movement
    df['vol_eom'] = ta.volume.ease_of_movement(df['gspc_high'],
                                               df['gspc_low'],
                                               df['gspc_close'],
                                               df['gspc_volume'],
                                               n=20,fillna=True)
  
    # Force Index
    df['vol_fm'] = ta.volume.force_index(df['gspc_close'],
                                         df['gspc_volume'],
                                         n=2,fillna=True)
  
    # Negative Volume Index
    df['vol_nvi'] = ta.volume.negative_volume_index(df['gspc_close'],
                                                    df['gspc_volume'],
                                                    fillna=True)
  
    # On-Balance Volume
    df['vol_obv'] = ta.volume.on_balance_volume(df['gspc_close'],
                                                df['gspc_volume'],
                                                fillna=True)
  
    # Volume-Price Trend
    df['vol_vpt'] = ta.volume.volume_price_trend(df['gspc_close'],
                                                 df['gspc_volume'],
                                                 fillna=True)
    
    ############### Volatility Feature Engineering
    
    #Average True Range
    df['atr_low'] = ta.volatility.average_true_range(df['gspc_high'],
                                                     df['gspc_low'],
                                                     df['gspc_close'],
                                                     n=23)
    
    df['atr_high'] = ta.volatility.average_true_range(df['gspc_high'],
                                                      df['gspc_low'],
                                                      df['gspc_close'],
                                                      n=37)

    ############### Target Creation #################
    
   
     # Determine daily market movement between Close and Close
    df['vix_move']  = (1 - df['vix_close']
                       .shift(1)/df['vix_close'])
  
    df['gspc_move'] = (1 - df['gspc_close']
                        .shift(1)/df['gspc_close'])
  
    ############## Handling Null Values ##################
    
    # Interpolating the Null Values for 30yr Treasury Bonds
    df['30 YR'] = (df['30 YR'].interpolate(method='spline',
                                           order=4))

    # Drop the rest
    df = df.dropna()
    
    # Reset the index for mistake free slicing
    df = df.reset_index().drop(columns = 'index')
  
    return df
  

In [8]:
df = get_data()

In [9]:
df.head()

,Date,vix_high,vix_low,vix_open,vix_close,gspc_high,gspc_low,gspc_open,gspc_close,gspc_volume,gspc_adj_close,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,30 YR,3m1s_inversion,3m2s_inversion,2s10s_inversion,2s30s_inversion,y/m,lag_index_level,lag_index_growth,fred,ism,gdp_cury,mom_ao,mom_mf,mom_rsi,mom_stoch,mom_st_sig,mom_tsi,mom_uo,mom_wr,vol_adi,vol_cmf,vol_eom,vol_fm,vol_nvi,vol_obv,vol_vpt,atr_low,atr_high,vix_move,gspc_move
0,1990-01-03,18.190001,18.190001,18.190001,18.190001,360.589996,357.890015,359.690002,358.760010,192330000,358.760010,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,8.04,True,False,False,False,1990-01,102.5,0.5,1748.0,47.2,6.4,0.000000,0.000000,0.000000,78.745764,89.372882,-100.000000,0.970766,-21.254236,9.368614e+07,0.264351,4.780031e-08,0.000000e+00,1000.000000,-192330000.0,-1.152928e+08,14.745670,14.952844,0.052227,-0.002592
1,1990-01-04,19.219999,19.219999,19.219999,19.219999,358.760010,352.890015,358.760010,355.670013,177000000,355.670013,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,8.04,True,False,False,False,1990-01,102.5,0.5,1748.0,47.2,6.4,0.000000,0.000000,0.000000,42.857244,73.867670,-100.000000,1.276824,-57.142756,-7.773132e+07,0.158710,-3.272694e-08,-6.001844e+07,991.387010,-369330000.0,-2.021776e+06,14.359771,14.707361,0.053590,-0.008688
2,1990-01-05,20.110001,20.110001,20.110001,20.110001,355.670013,351.350006,355.670013,352.200012,158530000,352.200012,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,8.06,False,False,False,False,1990-01,102.5,0.5,1748.0,47.2,6.4,0.000000,0.000000,0.000000,9.199210,43.600739,-100.000000,1.365730,-90.800790,-1.054926e+08,-0.017113,-4.284622e-08,2.217279e+08,981.714803,-527860000.0,-3.071156e+06,13.923259,14.426622,0.044257,-0.009852
3,1990-01-08,20.260000,20.260000,20.260000,20.260000,354.239990,350.540009,352.200012,353.790009,140110000,353.790009,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,8.09,False,False,False,False,1990-01,102.5,0.5,1748.0,47.2,6.4,0.000000,16.740532,21.133305,32.338348,28.131601,-99.956827,1.722127,-67.661652,9.885240e+06,0.113517,-3.952889e-08,6.935338e+07,986.146725,-387750000.0,-9.141340e+05,13.478769,14.136713,0.007404,0.004494
4,1990-01-09,22.200001,22.200001,22.200001,22.200001,354.170013,349.609985,353.829987,349.619995,155210000,349.619995,7.80,7.82,7.78,7.91,7.94,7.92,8.05,8.02,8.10,True,False,False,False,1990-01,102.5,0.5,1748.0,47.2,6.4,-0.210667,14.132502,12.889905,0.091164,13.876240,-99.925607,1.714533,-99.908836,-4.849819e+07,-0.061207,-3.456108e-08,8.565657e+06,986.146725,-542960000.0,-1.196890e+06,13.090998,13.877883,0.087387,-0.011927


In [10]:
start_date = datetime.datetime(2008, 12, 31)

df = df[df['Date'] > start_date]

In [11]:
Amazon = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Amazon.csv'
Apple = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Apple.csv'
BOFA = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/BOFA.csv'
Berkshire = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Berkshire.csv'
Google = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Google.csv'
JPM = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/JPM.csv'
Johnson = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Johnson.csv'
Mastercard = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Mastercard.csv'
Microsoft = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Microsoft.csv'
Proctor = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/Proctor.csv'
Visa = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/VISA.csv'
Boeing = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/ba.csv'
Citi = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/c.csv'
Comcast = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/cmcsa.csv'
Cisco = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/csco.csv'
Disney = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/dis.csv'
HD = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/hd.csv'
Intel = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/intc.csv'
Coke = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/ko.csv'
Mcdonalds = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/mcd.csv'
Merck = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/mrk.csv'
Pepsi = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/pep.csv'
Pfizer = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/pfe.csv'
ATT = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/t.csv'
UNH = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/unh.csv'
Verizon = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/vz.csv'
Wells = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/wfc.csv'
Walmart = 'https://raw.githubusercontent.com/equity-risk-ratings/Data-Science/master/wmt.csv'

In [12]:
Amazon = pd.read_csv(Amazon)
Apple = pd.read_csv(Apple)
BOFA = pd.read_csv(BOFA)
Berkshire = pd.read_csv(Berkshire)
Google = pd.read_csv(Google)
JPM = pd.read_csv(JPM)
Johnson = pd.read_csv(Johnson)
Mastercard = pd.read_csv(Mastercard)
Microsoft = pd.read_csv(Microsoft)
Proctor = pd.read_csv(Proctor)
Visa = pd.read_csv(Visa)
Boeing = pd.read_csv(Boeing)
Citi = pd.read_csv(Citi)
Comcast = pd.read_csv(Comcast)
Cisco = pd.read_csv(Cisco)
Disney = pd.read_csv(Disney)
HD = pd.read_csv(HD)
Intel = pd.read_csv(Intel)
Coke = pd.read_csv(Coke)
Mcdonalds = pd.read_csv(Mcdonalds)
Merck = pd.read_csv(Merck)
Pepsi = pd.read_csv(Pepsi)
Pfizer = pd.read_csv(Pfizer)
ATT = pd.read_csv(ATT)
UNH = pd.read_csv(UNH)
Verizon = pd.read_csv(Verizon)
Wells = pd.read_csv(Wells)
Walmart = pd.read_csv(Walmart)

In [13]:
Amazon['date'] = pd.to_datetime(Amazon['date'], yearfirst = True)
Apple['date'] = pd.to_datetime(Apple['date'], yearfirst = True)
BOFA['date'] = pd.to_datetime(BOFA['date'], yearfirst = True)
Berkshire['date'] = pd.to_datetime(Berkshire['date'], yearfirst = True)
Google['date'] = pd.to_datetime(Google['date'], yearfirst = True)
JPM['date'] = pd.to_datetime(JPM['date'], yearfirst = True)
Johnson['date'] = pd.to_datetime(Johnson['date'], yearfirst = True)
Mastercard['date'] = pd.to_datetime(Mastercard['date'], yearfirst = True)
Microsoft['date'] = pd.to_datetime(Microsoft['date'], yearfirst = True)
Proctor['date'] = pd.to_datetime(Proctor['date'], yearfirst = True)
Visa['date'] = pd.to_datetime(Visa['date'], yearfirst = True)


### Dateframe ###

- change columns
- merge
- clean data

In [14]:
df = df.rename(columns = {'Date': 'date'})

In [15]:
df['date'] = pd.to_datetime(df['date'])

df['date'].dtypes

dtype('<M8[ns]')

#### MERGE ####

In [ ]:
df = pd.merge(df, Amazon, how='inner', on = 'date')


In [17]:
df.head(2)

,date,vix_high,vix_low,vix_open,vix_close,gspc_high,gspc_low,gspc_open,gspc_close,gspc_volume,gspc_adj_close,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,30 YR,3m1s_inversion,3m2s_inversion,2s10s_inversion,2s30s_inversion,y/m,lag_index_level,lag_index_growth,fred,ism,gdp_cury,mom_ao,mom_mf,mom_rsi,mom_stoch,mom_st_sig,mom_tsi,mom_uo,mom_wr,vol_adi,vol_cmf,vol_eom,vol_fm,vol_nvi,vol_obv,vol_vpt,atr_low,atr_high,vix_move,gspc_move,Close,Volume
0,2009-01-02,39.820000,36.880001,39.580002,39.189999,934.729980,899.349976,902.989990,931.799988,4048270000,931.799988,0.08,0.28,0.40,0.88,1.14,1.72,2.07,2.46,2.83,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,19.188677,54.373563,70.641345,96.485977,77.194122,-9.042839,68.861605,-3.514023,4.693293e+09,0.090444,1.908986e-08,1.730653e+10,922.503526,2.932508e+11,1.870399e+08,36.975585,39.464797,-0.020669,0.03064,54.36,7296400
1,2009-01-05,40.220001,38.299999,39.240002,39.080002,936.630005,919.530029,929.169983,927.450012,5413910000,927.450012,0.14,0.32,0.43,0.78,1.08,1.67,2.07,2.49,3.00,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,28.948385,45.179753,67.842757,88.461547,87.278816,-8.439898,70.776905,-11.538453,2.978832e+09,0.113486,1.931402e-08,3.003149e+10,922.503526,2.878369e+11,1.026839e+08,36.111428,38.860343,-0.002815,-0.00469,54.06,9509800


#### Feature Engineering ####

In [25]:
# let's create a % change column and see if there are zeros in it

df['%_change'] = df['Close'].pct_change()

df['%_change'].head(10)

0         NaN
1   -0.005519
2    0.061043
3   -0.020223
4    0.017082
5   -0.028866
6   -0.064673
7   -0.009052
8   -0.057532
9    0.060837
Name: %_change, dtype: float64

In [26]:
df['close_move'] = (1 - df['Close']
                       .shift(1)/df['Close'])

In [28]:
# cols = ["Weight","Height","BootSize","SuitSize","Type"]
df['close_move'] = df['close_move'].replace({0:np.nan})

In [32]:
df = df.dropna()

In [34]:
df.shape

(2506, 53)

##### ENGINEER RATIO #####

In [35]:
df['ratio'] = abs(df['vix_move']/df['close_move'])

In [37]:
df['ratio'].describe()

count    2506.000000
mean       14.328237
std        73.620193
min         0.009413
25%         1.619702
50%         3.871668
75%         9.013867
max      2312.142278
Name: ratio, dtype: float64

#### DEFINE FEATURES ####

In [38]:
df.head(2)

,date,vix_high,vix_low,vix_open,vix_close,gspc_high,gspc_low,gspc_open,gspc_close,gspc_volume,gspc_adj_close,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,30 YR,3m1s_inversion,3m2s_inversion,2s10s_inversion,2s30s_inversion,y/m,lag_index_level,lag_index_growth,fred,ism,gdp_cury,mom_ao,mom_mf,mom_rsi,mom_stoch,mom_st_sig,mom_tsi,mom_uo,mom_wr,vol_adi,vol_cmf,vol_eom,vol_fm,vol_nvi,vol_obv,vol_vpt,atr_low,atr_high,vix_move,gspc_move,Close,Volume,%_change,close_move,ratio
1,2009-01-05,40.220001,38.299999,39.240002,39.080002,936.630005,919.530029,929.169983,927.450012,5413910000,927.450012,0.14,0.32,0.43,0.78,1.08,1.67,2.07,2.49,3.00,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,28.948385,45.179753,67.842757,88.461547,87.278816,-8.439898,70.776905,-11.538453,2.978832e+09,0.113486,1.931402e-08,3.003149e+10,922.503526,2.878369e+11,1.026839e+08,36.111428,38.860343,-0.002815,-0.004690,54.06,9509800,-0.005519,-0.005549,0.507201
2,2009-01-06,39.330002,37.340000,38.060001,38.560001,943.849976,927.280029,931.169983,934.700012,5392620000,934.700012,0.14,0.31,0.45,0.80,1.10,1.68,2.07,2.51,3.04,False,False,False,False,2009-01,167.2,1.2,545.0,36.4,-0.8,41.743648,53.586917,70.119271,89.456134,91.467886,-7.810315,68.329414,-10.543866,-9.619383e+08,0.052175,2.327680e-08,3.898648e+09,929.714859,2.932296e+11,1.688076e+07,35.261798,38.257900,-0.013485,0.007756,57.36,11080100,0.061043,0.057531,0.234402


In [39]:
to_drop = ['date', 'y/m', 'mom_wr', 'gspc_adj_close']

target = 'ratio'

features = (df.drop(columns = target)
           .drop(columns = to_drop)
           .columns)

In [40]:
X = df[features]
y = df['ratio']

X_train = X[:1600]
y_train = y[:1600]
X_test = X[1600:]
y_test = y[1600:]


X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1600, 49), (1600,), (906, 49), (906,))

In [42]:
!pip install xgboost

    100% |████████████████████████████████| 142.8MB 336kB/s eta 0:00:01��████████████████▌        | 104.7MB 55.0MB/s eta 0:00:01
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [46]:
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate 

model   = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                        colsample_bytree=1, gamma=0, learning_rate=0.13, max_delta_step=0,
                        max_depth=8, min_child_weight=.8, missing=None, n_estimators=180,
                        n_jobs=-1, objective='reg:squarederror',random_state=42, reg_alpha=0, 
                        reg_lambda=1, scale_pos_weight=1,seed=42, silent=True, subsample=1, 
                        eval_metric='rmse', normalize_type='forest')

scores =cross_validate(model,X_train,y_train,
                      scoring = 'neg_mean_absolute_error',
                      cv=5, return_train_score=True,
                       return_estimator=True)

pd.DataFrame(scores)

,fit_time,score_time,estimator,test_score,train_score
0,1.633146,0.008229,"XGBRegressor(base_score=0.5, booster='gbtree',...",-46.690393,-0.075480
1,1.618110,0.008978,"XGBRegressor(base_score=0.5, booster='gbtree',...",-7.915118,-0.049866
2,1.635173,0.008195,"XGBRegressor(base_score=0.5, booster='gbtree',...",-7.297891,-0.067565
3,1.623187,0.008208,"XGBRegressor(base_score=0.5, booster='gbtree',...",-5.620049,-0.084549
4,1.662333,0.008390,"XGBRegressor(base_score=0.5, booster='gbtree',...",-10.070650,-0.046649
